# Conexión por SSH

In [9]:
import paramiko
import getpass

host = "158.251.88.197"
port = 22
username = "gonzalo"
password = getpass.getpass()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

# Ejecución de comandos

Implementé un *servidor MCLP*, el cual escucha instrucciones para generar bins.

El siguiente código lanza el servidor en el background, carga una instancia y lo deja escuchando peticiones por el puerto 8080

In [10]:
transport = ssh.get_transport()
channel = transport.open_session()
idClass = 1
boxSize = 100
idInstance = 2
channel.exec_command("killall BASIC_BSG_MCLP; ./mclp/Metasolver/BASIC_BSG_MCLP mclp/Metasolver/problems/mclp/benchs/class"+str(idClass)+"/"+str(boxSize)+".txt -i "+str(idInstance))

Se carga la instancia asociada a su clase y numero de cajas

In [26]:
def getInstance(filePath, idInstance):
    boxesinfo = []
    binDim = []
    boxesFile = open(filePath,'r')
    instanceIndex = 0
    countBox = 0
    totalBox = 0

    for line in boxesFile.readlines():
        info = line.strip().split()

        if(instanceIndex < idInstance and len(info) == 4):
            instanceIndex+=1
            totalBox = int(info[0])
        
        elif(instanceIndex == idInstance):
            #La instancia que se esta buscando
            if(len(info) == 4):
                instanceIndex+=1
                totalBox = int(info[0])
                binDim = [int(info[i]) for i in range(1, len(info))]
            
            elif(len(info) == 3 and countBox < totalBox):
                countBox+=1    
                b = [int(info[i]) for i in range(0, len(info))]
                b.sort()
                boxesinfo.append(b)
    
    return boxesinfo

path = '../benchs/class1/100.txt'
instance = 2
x = getInstance(path, instance)

100


**Generación de conjunto inicial de bins con BSG**

La siguiente función genera la lista inicial de bins. Cada línea representa un bin indicando el porcentaje de llenado y los ids de sus cajas.

Es necesario modificarla para que en vez de imprimir el contenido de los bins, retorne una estructura adecuada.

In [12]:
def generate_bins():
    stdin, stdout, stderr = ssh.exec_command("echo generate_bins | netcat localhost 8080")
    flag = False
    bins = []
    for line in stdout.readlines():
        if "BINS:" in line: 
            flag = True
            continue
        if flag:    
            #Modificar igual que generate_bin
            line = line.strip().split()
            newBin = [ int(line[x]) for x in range(1,len(line)) ]
            bins.append(newBin)
    return bins


**Construcción de bin priorizando cajas (BSG)**

In [25]:
import paramiko
import getpass

def generate_bin(boxes, new_box):
    leftboxes = None
    boxes.append(new_box)
    boxes_str = [str(box) for box in boxes]
    pboxes_str = [str(pbox) for pbox in new_box]    
    stdin, stdout, stderr = ssh.exec_command("echo generate_bin " + " ".join(boxes_str) + " -1 " + " ".join(pboxes_str) +  " -2 | netcat localhost 8080")
    for line in stdout.readlines(): 
        #ajustar linea para almacenar los conjuntos de cajas
        print(line.strip())
    
    
    #primera linea boxes 
    #segunda linea left boxes
    boxes.remove(new_box)
    
    return boxes, leftboxes

host = "158.251.88.197"
port = 22
username = "gonzalo"
password = getpass.getpass()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

bins, ___ = generate_bin([0,1,8,53,45],[45])
print(bins)

[0, 1, 8, 53, 45]


Faltaría:

* Cargar la instancia en python (cajas con sus dimensiones) (Listo)
* Implementar algoritmo. (Listo)

A grandes rasgos el algoritmo hace los siguiente:
1. Generación de bins iniciales usando **BSG**
2. Selección de bin a desarmar y almacenar cajas en $C$
3. Mientras $C$ no quede vacío o máximo de iteraciones:

   1. Seleccionar caja $c$ de $C$
   2. Seleccionar bin de destino $B$
   3. Usar **BSG** para generar bin $B'$ usando cajas $B \cup \{c\}$, priorizando $c$. Es posible que **BSG** retorne conjunto de cajas residuales $R$
   4. Si $R$ es mejor que $c$, $B$ se reemplaza por $B'$ en el conjunto de bins y $C \gets C \cup R$
   
 4. Volver a 2 (seleccionar otro bin para desarmar)

In [14]:
def solve(solver, s0, swapIter):
    cantBoxes = s0.nbLeftBoxes
    maxVolBin = s0.getContVol
    boxes = set()
    bestBins = None
    bins = None

    for box in s0.boxes:
        boxes.add(box)
    
    #Llamada a BSG
    bins = generador()
    
    firstSol = len(bins)
    for i in range(swapIter):
        toReduce = 0 #Que haga referencia al indice del bin
        bins = reduceBins(bins, toReduce)
    
    return len(bins)


In [15]:
import copy as cp

def reduceBins(bins, toReduce):

    lastBins = cp.deepcopy(bins)
    breakedBin = bins.pop(toReduce)

    while(len(breakedBin) == 0 or iter <100):

        pivBin = popRandomBin(bins)    
        complexBox, evalComplexBox = getMostComplexBox(breakedBin)
        #Metodo de llamada al BSG
        boxes, leftBoxes = generate_bin(pivBin, complexBox)
        evalCB = calculateComplex()

        #ENFASIS: CUANDO ACEPTAR EL CAMBIO 

        if( getMostComplexBox(leftBoxes)[1] < evalComplexBox and vol(leftBoxes) <= vol([complexBox]) ):
            
            #Se inserta el nuevo bin dentro del conjunto de bins
            bins.append(boxes)
            #Se elimina (desde el conjunto de cajas a repartir) la caja insertada en el nuevo bin 
            breakedBin.remove(complexBox)
           
            #Se agregan las cajas que se cambiaron
            breakedBin += leftBoxes

        else:
            bins.append(pivBin)

    #Regresar al estado anterior si 
    if(len(breakedBin) > 0):
        return lastBins
    #Sino
    else: 
        return bins
        

In [16]:
def calculateComplex(idBox, dim):
    #Revisar metodo sort
    dim = dim.sort()
    x,y,z = dim
    expo = 1
    vol = x*y*z

    return ( ( (x/z)*(x/y)*(y/z) )**expo ) * vol

In [17]:
def getMostComplexBox(bin):
    idBox = None
    eval = 0
    
    for b in bin:
        boxShape = None #Cargar instancia
        currentEval = calculateComplex(b,boxShape)    
        if (idBox is None):
            idBox = b
            eval = currentEval
        else:
            if(currentEval > eval):
                idBox = b
                eval = currentEval
    return idBox, eval

In [18]:
import random as rand

def popRandomBin(bins):
    rPos = rand.random()%len(bins)
    theBin = bins.pop(rPos)
    return theBin

Para cerrar el servidor MCLP

In [19]:
def close_mclp_server():
    stdin, stdout, stderr = ssh.exec_command("echo END | netcat localhost 8080")
    print(stdout.readlines()[0])
    
close_mclp_server()

Ending MCLP server


## Aplicando Orientacion de objetos

In [20]:
class box:
    def __init__(self, dim, id):
        self.idBox = id
        self.x, self.y, self.z = dim
        self.vol = x*y*z
        self.complex = None
    
    def calculateComplex(self):
        expo = 1
        self.complex = (((self.x/self.z)*(self.x/self.y)*(self.y/self.z)))**expo) * self.vol
        

class bin:
    def __init__(self, boxes):
        self.boxes = boxes
        self.

    def getMostComplex(self):
        idBox = None
        eval = 0
        for b in self.boxes:
            if(idBox is None):
                idBox = b.idBox
            else:
                if(b.complex > eval):
                    idBox = b.idBox
                    eval = b.complex
        return idBox, eval    

    def getBox(self, idBox):
        for b in self.boxes:
            if (idBox == b.idBox):
                return b 
            else:
                continue   

    def getAllIndex(self):
        idBoxes = [ b.idBox for b in self.boxes ]
        return idBoxes


SyntaxError: unmatched ')' (<ipython-input-20-e27b311788e1>, line 10)